<a href="https://colab.research.google.com/github/SawadaSyahmi/5GNRcustomEnv/blob/main/5G_NR_ENV_PPO_MLP_policy_Custom_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PPO
!pip install stable-baselines3[extra]
!pip install gym
!pip install keras
!pip install keras-rl2

# 1. Import Dependencies

In [ ]:
import gym 
from gym import Env
from gym import spaces, logger
from gym.spaces import Discrete, Box, Dict, MultiBinary, MultiDiscrete 
import numpy as np
import random
from gym.utils import seeding
import os
from stable_baselines3 import PPO
from stable_baselines3 import A2C
from stable_baselines3 import DQN
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy
import math
from google.colab import drive
# drive.mount('/content/drive')

# models_dir = "C:/Users/FARIS SYAHMI/Desktop/Kerja/5GReinforceLearning/5NRmodel_logs/model/PPO"
# logdir = "C:/Users/FARIS SYAHMI/Desktop/Kerja/5GReinforceLearning/5NRmodel_logs/logs"

# models_dir = "/content/drive/MyDrive/Colab Notebooks/5GNRmodel_logs/models/PPO"
# logdir = "/content/drive/MyDrive/Colab Notebooks/5GNRmodel_logs/logs"

models_dir = "/model/DQN"
logdir = "/logs"

if not os.path.exists(models_dir):
     os.makedirs(models_dir)

if not os.path.exists(logdir):
     os.makedirs(logdir)

## 2. Building an Environment

In [ ]:
class environment(Env):

########### init method or constructor #############################
    def __init__(self):
        # env config

        self.np_random = None
        self.ULDLratio = 0.9   #Uplink ratio
        self.ULDLratio2 = 0.9   #Uplink ratio
        self.ULDLratio3 = 0.9   #Uplink ratio
        self.UE = 12
        self.max_tx_power_interference = 40 #in watts
        self.subcarrierSpacing1 = 4
        self.subcarrierSpacing2 = 4
        self.subcarrierSpacing2 = 4
        self.txPower = 40
        self.UEtxPower = dbW_watts(43-30)   ##43bdm to watt
        self.f_c = 700e6  # Hz
        self.cell_radius = 150  # in meters.
        self.inter_site_distance =  self.cell_radius / 2.
        self.FSPL = False
        self.x_bs_1, self.y_bs_1 = 0, 0
        self.x_bs_2, self.y_bs_2 = self.inter_site_distance+self.radius, 0
        self.guardband1 = 120
        self.guardband2 = 120

        # RL config
        self.seed(seed=10)
        self.state = None
        self.num_actions = 512
        self.step_count = 0  # which step
        self.reward_min = -500
        self.reward_max = 100000000
        self.length = 100

        bounds_lower = np.array([
            1,
            0,
            0.1,
            0.1,
            0.1,
            0,
            1,
            1,
            0,
            0,
            1,
            1,
            1,
            ])

        bounds_upper = np.array([
            self.UE,
            self.UEtxPower,
            self.ULDLratio,
            self.ULDLratio2,
            self.ULDLratio3,                     
            self.inter_site_distance + self.cell_radius,
            self.txPower,
            self.max_tx_power_interference,
            self.guardband1, 
            self.guardband2, 
            self.subcarrierSpacing1,
            self.subcarrierSpacing2,
            self.subcarrierSpacing3,       
            ])

        self.action_space = spaces.Discrete(self.num_actions)  # action size is here
        self.observation_space = spaces.Box(bounds_lower, bounds_upper,shape=(7,),
                                            dtype=np.float32)  # spaces.Discrete(2) # state size is here 
        int_ratio,int_radius,int_tx,int_itx,int_gb,int_ss1,int_ss2 = self.observation_space.sample()                                    
        # self.intSINR = self.SINR(int_tx,int_itx,int_ratio,int_gb,int_ss1,int_ss2)                                    
        # self.intBitrate = self.bitrate(int_ratio,int_ss1)
        self.intBitrate = 10
        self.intSINR = dbW_watts(40) #db to watt
        # Set start state
        self.state = None
        # Set shower length


########### seeding #############################

    def seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]        

########### step action #############################

    def step(self, action):
        

        # Apply action
        # 0 -1 = -1 temperature
        # 1 -1 = 0 
        # 2 -1 = 1 temperature 
        (UE,UEtxpower,ULDLratio,ULDLratio2,ULDLratio3, x_cell_radius, txPower, interferencePower, guardband, guardband2, subcarrierSpacing1, subcarrierSpacing2,subcarrierSpacing3) = self.state
        #print(action)
        power_tx = action & 0b00000001                # 1 power up, 0 power down
        ratio = (action & 0b00000010)             >> 1
        subcarrier1 = (action & 0b00000100)       >> 2
        gb = (action & 0b00001000)                >> 3
        subcarrier2 = (action & 0b00010000)       >> 4
        intpower = (action & 0b000100000)          >> 5
        subcarrier3 = (action & 0b001000000)       >> 6 
        ratio2 = (action & 0b00010000000)             >> 7
        ratio3 = (action & 0b00100000000)             >> 8
        gb2 = (action & 0b0100000000)                >> 9

         

                  

        self.step_count += 1    

        if (power_tx == 1):
            txPower *= 10**(1/10.)
        else:
            txPower *= 10**(-1/10.)

        if (intpower == 1):
            interferencePower *= 10**(1/10.)
        else:
            interferencePower *= 10**(-1/10.)         

        if (ratio == 1):
            ULDLratio  = ULDLratio + 0.1
        else:
            ULDLratio = ULDLratio - 0.1
        
        if (ratio2 == 1):
            ULDLratio2  = ULDLratio2 + 0.1
        else:
            ULDLratio2 = ULDLratio2 - 0.1
        
        if (ratio3 == 1):
            ULDLratio3  = ULDLratio3 + 0.1
        else:
            ULDLratio3 = ULDLratio3 - 0.1        
            
        if (subcarrier1 == 1):
            subcarrierSpacing1 = subcarrierSpacing1 + 1
        else:
            subcarrierSpacing1 = subcarrierSpacing1 - 1

        if (subcarrier2 == 1):
            subcarrierSpacing2 = subcarrierSpacing2 + 1
        else:
            subcarrierSpacing2 = subcarrierSpacing2 - 1

        if (subcarrier3 == 1):
            subcarrierSpacing3 = subcarrierSpacing3 + 1
        else:
            subcarrierSpacing3 = subcarrierSpacing3 - 1                 

        if (gb == 1):
            guardband = guardband + 15
        else:
            guardband = guardband - 15

        if (gb2 == 1):
            guardband2 = guardband2 + 15
        else:
            guardband2 = guardband2 - 15

        guardband1 = [guardband,guardband2]
            
                                

        
        # Reduce shower length by 1 second
        self.length -= 1 
        # print(self.length)


        #divide UE into link based on demand
        absDistanceBS1,absDistanceBS2,UEtrans,UEdemand = UEposition(UE)
        Ue_mmtc_UL = 0 
        Ue_EMBB_UL = 0
        Ue_URLLC_UL = 0 
        Ue_mmtc_DL = 0
        Ue_EMBB_DL = 0
        Ue_URLLC_DL = 0
        mm_D_bs1_UL = []
        mm_D_bs2_UL =[]
        mm_D_bs1 = []
        mm_D_bs2 =[]
        ur_D_bs1_UL = []
        ur_D_bs2_UL =[]
        ur_D_bs1 = []
        ur_D_bs2 =[]
        em_D_bs1_UL = []  
        em_D_bs2_UL =[]
        em_D_bs1 = []
        em_D_bs2 =[]


        for i in range(0,UE):
          if UEdemand[i] <= 20:

            if UEtrans[i] == 0:
              mm_D_bs1_UL.append(Ue_mmtc_UL)
              mm_D_bs2_UL.append(Ue_mmtc_UL)
              mm_D_bs1_UL[Ue_mmtc_UL] = absDistanceBS1[i]
              mm_D_bs2_UL[Ue_mmtc_UL] = absDistanceBS2[i]
              Ue_mmtc_UL += 1
        
            else:     
              mm_D_bs1.append(Ue_mmtc_DL)
              mm_D_bs2.append(Ue_mmtc_DL)
              mm_D_bs1[Ue_mmtc_DL] = absDistanceBS1[i]
              mm_D_bs2[Ue_mmtc_DL] = absDistanceBS2[i]
              Ue_mmtc_DL += 1

          elif UEdemand[i]>20:
            if UEtrans[i] == 0:
              ur_D_bs1_UL.append(Ue_URLLC_UL)
              ur_D_bs2_UL.append(Ue_URLLC_UL)
              ur_D_bs1_UL[Ue_URLLC_UL] = absDistanceBS1[i]
              ur_D_bs2_UL[Ue_URLLC_UL] = absDistanceBS2[i]
              Ue_URLLC_UL += 1

            else:
              ur_D_bs1.append(Ue_URLLC_DL)
              ur_D_bs2.append(Ue_URLLC_DL)
              ur_D_bs1[Ue_URLLC_DL] = absDistanceBS1[i]
              ur_D_bs2[Ue_URLLC_DL] = absDistanceBS2[i]
              Ue_URLLC_DL += 1

          elif UEdemand[i]>50:
            if UEtrans[i] == 0:
              em_D_bs1_UL.append(Ue_EMBB_UL)
              em_D_bs2_UL.append(Ue_EMBB_UL)      
              em_D_bs1_UL[Ue_EMBB_UL] = absDistanceBS1[i]
              em_D_bs2_UL[Ue_EMBB_UL] = absDistanceBS2[i]
              Ue_EMBB_UL += 1

            else:
              em_D_bs1.append(Ue_EMBB_DL)
              em_D_bs2.append(Ue_EMBB_DL)
              em_D_bs1[Ue_EMBB_DL] = absDistanceBS1[i]
              em_D_bs2[Ue_EMBB_DL] = absDistanceBS2[i]
              Ue_EMBB_DL += 1    
                 

        # Calculate reward link 1 URLLC
        SINRupdated1_UL, SEupdated1_UL,Throughput_updated1_UL= self.SINR(Ue_URLLC_UL,ur_D_bs1_UL,ur_D_bs2_UL,UEtxpower,interferencePower,ULDLratio,guardband,subcarrierSpacing1,subcarrierSpacing2,0)
        intBitrate1 = self.bitrate(ULDLratio, subcarrierSpacing1)
        

        if SINRupdated1_UL >= 0.9*self.intSINR: 
            reward1 = 4 
        elif SINRupdated1_UL >= 0.8*self.intSINR: 
            reward1 = 3
        elif SINRupdated1_UL >= 0.7*self.intSINR: 
            reward1 = 2
        elif SINRupdated1_UL >= 0.6*self.intSINR: 
            reward1 = 1           
        else: 
            reward1 = -10 

        if Throughput_updated1_UL >= 0.9*self.intBitrate1: 
            reward2 = 4 
        elif Throughput_updated1_UL >= 0.8*self.intBitrate1: 
            reward2 = 3 
        elif Throughput_updated1_UL >= 0.7*self.intBitrate1: 
            reward2 = 2 
        elif Throughput_updated1_UL >= 0.6*self.intBitrate1: 
            reward2 = 1            
        else: 
            reward2 = -10 

        # Calculate reward link 2 EMBB
        SINRupdated2_UL, SEupdated2_UL,Throughput_updated2_UL= self.SINR(Ue_EMBB_UL,em_D_bs1_UL,em_D_bs2_UL,UEtxpower,interferencePower,ULDLratio2,guardband1,0,subcarrierSpacing2,subcarrierSpacing3)
        intBitrate2 = self.bitrate(ULDLratio, subcarrierSpacing2)
        

        if SINRupdated2_UL >= 0.9*self.intSINR: 
            reward3 = 4 
        elif SINRupdated2_UL >= 0.8*self.intSINR: 
            reward3 = 3
        elif SINRupdated2_UL >= 0.7*self.intSINR: 
            reward3 = 2
        elif SINRupdated2_UL >= 0.6*self.intSINR: 
            reward3 = 1           
        else: 
            reward3 = -10 

        if Throughput_updated2_UL >= 0.9*self.intBitrate2: 
            reward4 = 4 
        elif Throughput_updated2_UL >= 0.8*self.intBitrate2: 
            reward4 = 3 
        elif Throughput_updated2_UL >= 0.7*self.intBitrate2: 
            reward4 = 2 
        elif Throughput_updated2_UL >= 0.6*self.intBitrate2: 
            reward4 = 1            
        else: 
            reward4 = -10 

        # Calculate reward link 3 MMTC
        
        SINRupdated3_UL, SEupdated3_UL,Throughput_updated3_UL= self.SINR(Ue_mmtc_UL,mm_D_bs1_UL,mm_D_bs2_UL,UEtxpower,interferencePower,ULDLratio3,guardband2,subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3)
        intBitrate3 = self.bitrate(ULDLratio, subcarrierSpacing3)
        

        if SINRupdated3_UL >= 0.9*self.intSINR: 
            reward5 = 4 
        elif SINRupdated3_UL >= 0.8*self.intSINR: 
            reward5 = 3
        elif SINRupdated3_UL >= 0.7*self.intSINR: 
            reward5 = 2
        elif SINRupdated3_UL >= 0.6*self.intSINR: 
            reward5 = 1           
        else: 
            reward5 = -10 

        if Throughput_updated3_UL >= 0.9*self.intBitrate3: 
            reward6 = 4 
        elif Throughput_updated3_UL >= 0.8*self.intBitrate3: 
            reward6 = 3 
        elif Throughput_updated3_UL >= 0.7*self.intBitrate3: 
            reward6 = 2 
        elif Throughput_updated_UL >= 0.6*self.intBitrate3: 
            reward6 = 1            
        else: 
            reward6 = -10


        #downlink reward

        SINRupdated1_DL, SEupdated1_DL,Throughput_updated1_DL= self.SINR(Ue_URLLC_DL,ur_D_bs1_DL,ur_D_bs2_DL,txPower,interferencePower,1-ULDLratio,guardband,subcarrierSpacing1,subcarrierSpacing2,0)
        intBitrate1 = self.bitrate(1-ULDLratio, subcarrierSpacing1)


        if SINRupdated1_DL >= 0.9*self.intSINR: 
            reward7 = 4 
        elif SINRupdated1_DL >= 0.8*self.intSINR: 
            reward7 = 3
        elif SINRupdated1_DL >= 0.7*self.intSINR: 
            reward7 = 2
        elif SINRupdated1_DL >= 0.6*self.intSINR: 
            reward7 = 1           
        else: 
            reward7 = -10 

        if Throughput_updated1_DL >= 0.9*self.intBitrate1: 
            reward8 = 4 
        elif Throughput_updated1_DL >= 0.8*self.intBitrate1: 
            reward8 = 3 
        elif Throughput_updated1_DL >= 0.7*self.intBitrate1: 
            reward8 = 2 
        elif Throughput_updated1_DL >= 0.6*self.intBitrate1: 
            reward8 = 1            
        else: 
            reward8 = -10 

        # Calculate reward link 2 EMBB
        SINRupdated2_DL, SEupdated2_DL,Throughput_updated2_DL= self.SINR(Ue_EMBB_DL,em_D_bs1_DL,em_D_bs2_DL,txPower,interferencePower,1-ULDLratio2,guardband1,0,subcarrierSpacing2,subcarrierSpacing3)
        intBitrate2 = self.bitrate(1-ULDLratio2, subcarrierSpacing2)


        if SINRupdated2_DL >= 0.9*self.intSINR: 
            reward9 = 4 
        elif SINRupdated2_DL >= 0.8*self.intSINR: 
            reward9 = 3
        elif SINRupdated2_DL >= 0.7*self.intSINR: 
            reward9 = 2
        elif SINRupdated2_DL >= 0.6*self.intSINR: 
            reward9 = 1           
        else: 
            reward9 = -10 

        if Throughput_updated2_DL >= 0.9*self.intBitrate2: 
            reward10 = 4 
        elif Throughput_updated2_DL >= 0.8*self.intBitrate2: 
            reward10 = 3 
        elif Throughput_updated2_DL >= 0.7*self.intBitrate2: 
            reward10 = 2 
        elif Throughput_updated2_DL >= 0.6*self.intBitrate2: 
            reward10 = 1            
        else: 
            reward10 = -10 

        # Calculate reward link 3 MMTC

        SINRupdated3_DL, SEupdated3_DL,Throughput_updated3_DL= self.SINR(Ue_mmtc_DL,mm_D_bs1_DL,mm_D_bs2_DL,txPower,interferencePower,1-ULDLratio3,guardband2,subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3)
        intBitrate3 = self.bitrate(1-ULDLratio3, subcarrierSpacing3)


        if SINRupdated3_DL >= 0.9*self.intSINR: 
            reward11 = 4 
        elif SINRupdated3_DL >= 0.8*self.intSINR: 
            reward11 = 3
        elif SINRupdated3_DL >= 0.7*self.intSINR: 
            reward11 = 2
        elif SINRupdated3_DL >= 0.6*self.intSINR: 
            reward11 = 1           
        else: 
            reward11 = -10 

        if Throughput_updated3_DL >= 0.9*self.intBitrate3: 
            reward12 = 4 
        elif Throughput_updated3_DL >= 0.8*self.intBitrate3: 
            reward12 = 3 
        elif Throughput_updated3_DL >= 0.7*self.intBitrate3: 
            reward12 = 2 
        elif Throughput_updated_DL >= 0.6*self.intBitrate3: 
            reward12 = 1            
        else: 
            reward12 = -10        



       reward = reward1 + reward2 + reward3 + reward4 + rewards5 + reward6 + reward7 + reward8 + reward9 + reward10 + rewards11 + reward12         
        
  
        # Check if shower is done
        if self.length <= 0: 
            done = True
            self.length = 100
        else:
            done = False
            

        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        self.state = (UE, UEtxpower, ULDLratio,ULDLratio2,ULDLratio3, x_cell_radius, txPower, interferencePower, guardband, guardband2, subcarrierSpacing1,subcarrierSpacing2,subcarrierSpacing3)
        # Return step information
        return self.state, reward, done, info

########### rendering #############################

    def render(self):
        # Implement viz
        pass

########### reset #############################

    def reset(self):
        # Initialize f_n of both cells
        self.state = [self.np_random.uniform(low=1,     high=self.UE),
                      self.np_random.uniform(low=1,     high=self.UEtxPower),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio2),
                      self.np_random.uniform(low=0.1,   high=self.ULDLratio3),
                      self.np_random.uniform(low=0,   high=self.inter_site_distance + self.cell_radius),
                      self.np_random.uniform(low=1,   high=self.txPower / 2),
                      self.np_random.uniform(low=1,   high=self.max_tx_power_interference / 2),
                      self.np_random.uniform(low=0,   high=self.guardband),
                      self.np_random.uniform(low=0,   high=self.guardband2),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing1),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing2),
                      self.np_random.uniform(low=1,   high=self.subcarrierSpacing3),
                      ]

        self.step_count = 0
        return np.array(self.state)


########### calculation #############################
    def SINR(self,UE,distanceBS1,distanceBS2,txpower,intpower,ratio,gb,ss1,ss2,ss3):
        
        gb1,gb2 = gb
        T = 290  # Kelvins
        B = 15000  # Hz
        k_Boltzmann = 1.38e-23
        noisePower = k_Boltzmann * T * B  # this is in Watts
        B = BW*1e6

        for i in range(0,UE,1)        
          receivedSinr = self.receivedPower(txpower,distanceBS1[i]) / (self.receivedPower(intpower,distanceBS2[i]) + noisePower + self.iniPower(gb1,ss1,ss2) + self.iniPower(gb2,ss2,ss3))
          UESinr.append(i)
          UESinr[i] = receivedSinr
          #print('receivedSinr Power = ' + str(receivedSinr))

        avgR_SINR = np.mean(UESinr)
        SE = np.log2(1 + avgR_SINR)
        Throughput = B*np.log2(avgR_SINR)*ratio
        #print('Spectrum Efficiency = ' + str(SE))

        return avgR_SINR,SE,Throughput

    def UEposition(numUE):
        radiusBS1 = 200
        heightBS = 80
        intersite = radiusBS1/2 + radiusBS1
        xbs1,ybs1 = 0,0
        xbs2,ybs2 = intersite,0
        distanceBS1 = []
        distanceBS2 = []
        absDistanceBS1 =[]
        absDistanceBS2 =[]
        UEtrans=[]
        UEdemand=[]
        x=[]
        y=[]

        for i in range(0,numUE,1):
          x.append(i)
          y.append(i)
          distanceBS1.append(i)
          distanceBS2.append(i)
          absDistanceBS1.append(i)
          absDistanceBS2.append(i)
          UEtrans.append(i)
          UEdemand.append(i)
          
          x[i] = random.randint(-radiusBS1,intersite+radiusBS1)
          y[i] = random.randint(-radiusBS1,radiusBS1)
          distanceBS1[i] = math.sqrt((x[i]-xbs1)**2 + (y[i]-ybs1)**2)
          distanceBS2[i] = math.sqrt((x[i]-xbs2)**2 + (y[i]-ybs2)**2)
          absDistanceBS1[i] = math.sqrt(distanceBS1[i]**2 + (heightBS)**2)
          absDistanceBS2[i] = math.sqrt(distanceBS2[i]**2 + (heightBS)**2)
          UEtrans[i] = random.randint(0,1)
          UEdemand[i] = random.randint(10,100) #mbps
        return absDistanceBS1,absDistanceBS2,UEtrans,UEdemand

    def receivedPower(self,txpower,distance):
        d_mainBS = distance
        txpower_db =  watts_dbW(txpower)
        txpower_dbm = dbW_dbm(txpower_db)

        if self.FSPL == True:
            pathloss = 20 * np.log10(d_mainBS) + 20 * np.log10(self.f_c) + 32.44 #in db
            #print('pathloss = ' + str(pathloss))
        else:
            f_c = self.f_c
            c = 3e8  # speed of light
            d = d_mainBS
            h_B = 20
            h_R = 1.5

            # COST231
            C = 3
            a = (1.1 * np.log10(f_c / 1e6) - 0.7) * h_R - (1.56 * np.log10(f_c / 1e6) - 0.8)
            pathloss = 46.3 + 33.9 * np.log10(f_c / 1e6) + 13.82 * np.log10(h_B) - a + (
                    44.9 - 6.55 * np.log10(h_B)) * np.log10(d / 1000.) + C # in db
            #print('pathloss = ' + str(pathloss))

        receivedPower_db = txpower_db - pathloss
        receivedPower =  dbW_watts(receivedPower_db)

        return receivedPower

    def bitrate(self,ratio,ss1):
        N_prb = 100
        ss = int(ss1)

        if ss == 1:
            N_prb = 270
        elif ss == 2:
            N_prb = 133
        elif ss == 3:
            N_prb = 65
        elif ss == 4:
            N_prb = 32


        J = 1  # carrier aggregation
        v = 1  # MIMO layers
        Q = 6  # modulation order
        Rmax = 0.92578125  # LDPC code
        f = 1  # scaling factor
        OH = 0.14  # overhead control
        ULDLratio = ratio
        T_ofdm = 0.0000357142857142852  # (10 ** -3) / (14 * 2^u)
        _bitrate = v * Q * f * Rmax * (12 * N_prb / T_ofdm) * (1 - OH)
        Mbitrate = _bitrate * (10 ** -6)
        Mbitrate = Mbitrate * ULDLratio

        #print('Max BitRate = ' + str(Mbitrate))
        return Mbitrate


    def iniPower(self,gb,ss1,ss2):

        ss1 = int(ss1)
        #print('Max ss1 = ' + str(ss1))
        ss2 = int(ss2)
        #print('Max ss2 = ' + str(ss2))
        inipower = []
        z1 = 240       


        if (ss1 == 1):
          Tn_bs1 = 1025
        elif (ss1 == 2):
          Tn_bs1 = 512
        elif (ss1 == 3):
          Tn_bs1 = 256
        elif (ss1 == 4):
          Tn_bs1 = 128
        else:
            print('Wrong Subcarrier Spacing')

        if (ss2 == 1):
          Tn_bs2 = 1052
        elif (ss2 == 2):
          Tn_bs2 = 512
        elif (ss2 == 3):
          Tn_bs2 = 256
        elif (ss2 == 4):
          Tn_bs2 = 128
        else:
            print('Wrong Subcarrier Spacing')

        if ss1==ss2 || ss1==0 || ss2==0:
          inipower = 0
        else:
          for i in range(1,z1):
            Tn_bs1 = 1025
            Tn_bs2 = 512
            n_cp = 1-0.07
            n_bs1 = round(Tn_bs1*n_cp) 
            n_bs2 = round(Tn_bs2*n_cp)         
            pt_bs2 = 20
            k = 100
            n2 = 2

            v1 = 0
            OFDM_overlap = n_bs1 / Tn_bs2
            gain = 1
            total_v = 0
            total_z = 0

            # a = pt_bs2 / n_bs2
            # b = gain / (n_bs2 * n_bs1)
            # c = np.sin(math.pi / n_bs2 * (z1 - v1 - gb) * OFDM_overlap * Tn_bs1)
            # d = np.sin(math.pi / n_bs2 * (z1 - v1 - gb))
            # e = np.absolute((c / d) ** 2)

            # g = np.sin(math.pi / n_bs2 * (z1 - v1 - gb)*Tn_bs1)  
            # i = np.absolute((g / d) ** 2)
            # j = OFDM_overlap * i

            if ss1 > ss2:
              a = pt_bs2 / n_bs2
              b = gain / (n_bs2 * n_bs1)
              c = np.sin(math.pi / n_bs2 * (z1 - v1 - gb) * n_bs1)
              d = np.sin(math.pi / n_bs2 * (z1 - v1 - gb))
              e = np.absolute((c / d) ** 2)
              inipower = a*(b * e)

            else:
              a = pt_bs2 / n_bs2
              b = gain / (n_bs2 * n_bs1)
              c = np.sin(math.pi / n_bs1 * (z1 - v1 - gb) * OFDM_overlap * Tn_bs2)
              d = np.sin(math.pi / n_bs1 * (z1 - v1 - gb))
              e = np.absolute((c / d) ** 2)

              g = np.sin(math.pi / n_bs1 * (z1 - v1 - gb)*Tn_bs2)  
              i = np.absolute((g / d) ** 2)
              j = OFDM_overlap * i
              inipower = a*(b * (e + j))     


            # for z1 in range(1, z+1):
            #     for v1 in range(v, 1, -1):
            #         if not z1 == v1:
            #             d = np.sin(math.pi / n_bs2 * (z1 - v1 - gb) * OFDM_overlap * n_bs1)
            #             e = np.sin(math.pi / n_bs2 * (z1 - v1 - gb))
            #             c = np.absolute((d / e) ** 2)
            #             f = OFDM_overlap * c
            #             b = gain / (n_bs2 * n_bs1) * (c + f)
            #             total_v = total_v + b
            #             print(total_v)
            #     total_z = total_z + total_v
            # inipower = a * total_z
            #print('ini Power = ' + str(inipower_b))

            ini_power.append(inipower)
          test2 = np.array(ini_power)
          total = np.sum(test2)
          total =  dbW_watts(total)

        return total

    def spectrum_efficiency(self):
        SE = np.log2(1 + self.SINR())
        #print('Spectrum Efficiency = ' + str(SE))
        return SE
    

In [ ]:
### unit conversion

def watts_dbW(power):
    dbW = 10*np.log10(power)
    return dbW

def dbW_dbm(dbW):
    dbm = dbW + 30
    return dbm

def dbW_watts(dbW):
    power = 10**(dbW/10)
    return power

def dbm_dbW(dbm):
    dbW = dbm - 30
    return dbW


In [ ]:
env=environment()

/usr/local/lib/python3.7/dist-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
  "Box bound precision lowered by casting to {}".format(self.dtype)


In [ ]:
env.observation_space.sample()

array([  0.24397692, 156.19821   ,   2.512165  ,  21.985403  ,
        53.383904  ,   1.6810949 ,   2.4046402 ], dtype=float32)

In [ ]:
env.reset()

## 3. Test Environment

In [ ]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
    print('Updated State:{}'.format(n_state))
env.close()

Episode:1 Score:4
Episode:2 Score:60
Episode:3 Score:36
Episode:4 Score:30
Episode:5 Score:-54


In [ ]:
env.close()

# 4. Train Model

In [ ]:
#log_path = os.path.join('Training', 'Logs')

In [ ]:
model = DQN("MlpPolicy", env, verbose=1, tensorboard_log=logdir)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [ ]:
total_timesteps = 40000
for i in range(30):
  model.learn(total_timesteps, reset_num_timesteps=False, tb_log_name="DQN")
  model.save(f"{models_dir}/{total_timesteps*i}")

# 5. Save Model

In [ ]:
tensorboard --logdir=logs

In [ ]:
model.save('PPO')

In [ ]:
evaluate_policy(model, env, n_eval_episodes=10, render=False)

### 6. Bash Terminal

In [ ]:
!pip install google-colab-shell
from google_colab_shell import getshell
getshell()